In [9]:
import boto3
import pandas as pd

table_name = 'beacons'
client = boto3.client('dynamodb')


In [16]:
start_timestamp = 1724234689000
end_timestamp   = 1724234689000 + 120000

In [11]:

response = client.query(
    TableName=table_name,
     KeyConditionExpression='#src = :tracker_1 AND timestamp_ms BETWEEN :start AND :end', 
    ExpressionAttributeNames={
        '#src': 'source'  # Assuming 'source' is the name of your primary partition key
    },
    ExpressionAttributeValues={
        ':tracker_1': {'S': 'traker_1'} ,
        ':start': {'S': str(start_timestamp)},
        ':end': {'S': str(end_timestamp)} 
    }
)

In [12]:
items = response['Items']
beacons = []
for item in items:
    beacons.append([item['timestamp_ms']['S'], item['source']['S'], item['rssi']['N'], item['mac']['S']]) 

dtype = {'timestamp_ms': 'int64', 'source': 'str', 'rssi': 'int64', 'mac': 'str'}
df = pd.DataFrame(beacons, columns=['timestamp_ms', 'source', 'rssi', 'mac']).astype(dtype)
print(" Unique MAC addresses: ", len(df['mac'].unique()))    


 Unique MAC addresses:  9


In [13]:
# print mac addresses for beacons inside the room and beacons outside the room
df_place = df.groupby(['mac']).max().sort_values(by='rssi', ascending=False)['rssi'] > -75
for mac in df_place.index:
    if df_place[mac]:
        print(f"beacon: {mac} is inside the room")
    else:
        print(f"beacon: {mac} is outside the room")

beacon: d0:36:dd:ef:4d:0c is inside the room
beacon: c4:8f:be:90:22:7a is inside the room
beacon: c1:67:43:fd:a0:9f is inside the room
beacon: c4:ea:b3:69:fc:30 is inside the room
beacon: ea:10:2f:0e:f1:c5 is inside the room
beacon: cf:45:f7:4c:89:d3 is inside the room
beacon: c0:28:5d:63:81:5c is inside the room
beacon: e3:31:dd:f0:33:52 is inside the room
beacon: c6:58:98:77:12:d3 is inside the room


In [14]:
df.groupby(['mac'])['rssi'].mean().sort_values(ascending=False)

mac
d0:36:dd:ef:4d:0c   -45.375000
c4:8f:be:90:22:7a   -52.250000
c4:ea:b3:69:fc:30   -54.000000
c1:67:43:fd:a0:9f   -55.300000
ea:10:2f:0e:f1:c5   -57.666667
cf:45:f7:4c:89:d3   -57.750000
c0:28:5d:63:81:5c   -63.818182
c6:58:98:77:12:d3   -65.880000
e3:31:dd:f0:33:52   -66.382353
Name: rssi, dtype: float64

In [15]:
df.groupby(['mac']).count().sort

,timestamp_ms,source,rssi
mac,,,
c0:28:5d:63:81:5c,11,11,11
c1:67:43:fd:a0:9f,40,40,40
c4:8f:be:90:22:7a,4,4,4
c4:ea:b3:69:fc:30,2,2,2
c6:58:98:77:12:d3,25,25,25
cf:45:f7:4c:89:d3,4,4,4
d0:36:dd:ef:4d:0c,24,24,24
e3:31:dd:f0:33:52,34,34,34
ea:10:2f:0e:f1:c5,3,3,3


In [ ]:
# commit 1